# instagram 크롤링

In [41]:
# num_scrolldown = '페이지 스크롤 횟수'
# 예시) num_scrolldown = 2
num_scrolldown = 5

# num_page = '크롤링할 게시글 수'
# 예시) num_page = 6
num_page = 30

In [42]:
from bs4 import BeautifulSoup as bs
import time
import random
from selenium import webdriver
import pandas as pd
from datetime import datetime

## 페이지 로그인

In [43]:
# 로그인 페이지 접속
driver = webdriver.Chrome('chromedriver')
driver.get("https://www.instagram.com/accounts/login/")
time.sleep(3)

# 창 최대화
driver.maximize_window()
time.sleep(1)

# 로그인 정보
username = 'gyeonggi_korea'
password = 'zhsxpscm123!'

element_id = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input')
element_id.send_keys(username)
element_password = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input')
element_password.send_keys(password)
time.sleep(1.5)

##로그인버튼 클릭
driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()
time.sleep(3)

<ipython-input-43-d49f7a4f36a8>:14: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_id = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[1]/div/label/input')
<ipython-input-43-d49f7a4f36a8>:16: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  element_password = driver.find_element_by_xpath('//*[@id="loginForm"]/div/div[2]/div/label/input')
<ipython-input-43-d49f7a4f36a8>:21: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  driver.find_element_by_css_selector('.sqdOP.L3NKy.y3zKF').click()


## 크롤링할 페이지 접속

In [44]:
# 크롤링할 페이지 접속
url = 'https://www.instagram.com/gyeonggi_korea/'
# webdriver 로드
driver.get(url)
driver.implicitly_wait(5)

# 페이지 크롤링
soup = bs(driver.page_source, 'lxml')
soup.title.text

'gyeonggi_korea(경기도)(@gyeonggi_korea) • Instagram 사진 및 동영상'

## 팔로워 수 수집

In [45]:
# 팔로워 수 수집
raw_num_follower = soup.find_all('span', 'g47SY')
num_follower = int(raw_num_follower[1].text.replace(',',''))
num_follower

9755

## 게시글 링크 수집

In [46]:
# 각 게시글 링크 크롤링
raw_post_link_list = []
for i in range(num_scrolldown):
    # 현재 페이지 크롤링
    driver.implicitly_wait(3)
    soup = bs(driver.page_source, 'lxml')

    # 게시물 링크 주소 수집
    row_post = soup.find_all('div', "Nnq7C weEfm")

    # 게시글 링크 3개씩 가져오기
    for row in row_post:
        # 한줄(row_post)에 3개의 게시글(a 태그) 존재
        a_tags = row.find_all('a')
        for one in a_tags:
            link = one.attrs['href']
            raw_post_link_list.append("https://www.instagram.com" + link)

    # 페이지 스크롤 다운
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep( random.uniform(1,3) )

# 수집한 링크 중 중복 링크 제거
post_link_list = []
for link in raw_post_link_list:
    if link not in post_link_list:
        post_link_list.append(link)

# 수집한 링크 개수 확인
print("수집한 링크의 개수 : ", len(post_link_list))
print("수집한 링크 주소")
print(post_link_list)

수집한 링크의 개수 :  60
수집한 링크 주소
['https://www.instagram.com/p/CbbxBIRvhi1/', 'https://www.instagram.com/p/CbXJ1rXPJdy/', 'https://www.instagram.com/p/CbRd0niv2Vl/', 'https://www.instagram.com/p/CbPIO4bPUQJ/', 'https://www.instagram.com/p/CbHbw-uB5B3/', 'https://www.instagram.com/p/Ca_cN1HPMlv/', 'https://www.instagram.com/p/Ca6eIqfPoTV/', 'https://www.instagram.com/p/Ca1mOngPNr7/', 'https://www.instagram.com/p/CarTSR8hpjk/', 'https://www.instagram.com/p/CamCGL3LSHX/', 'https://www.instagram.com/p/Cag862TrWuX/', 'https://www.instagram.com/p/CaWWan2BoZc/', 'https://www.instagram.com/p/CaRgX_svAjY/', 'https://www.instagram.com/p/CaG5pIbv7MV/', 'https://www.instagram.com/p/CZ9rGIHP423/', 'https://www.instagram.com/p/CZx-sLRPoq_/', 'https://www.instagram.com/p/CZs1BDAqmYF/', 'https://www.instagram.com/p/CZmf9rhPnGY/', 'https://www.instagram.com/p/CZjB2LzP7mn/', 'https://www.instagram.com/p/CZMv78CvpIk/', 'https://www.instagram.com/p/CZL99vIDzxe/', 'https://www.instagram.com/p/CZGj2IOvVIf/', 'htt

## 데이터 크롤링

In [47]:
date_list = []
title_list = []
num_view_list = []
num_like_list = []
num_comment_list = []
num_share_list = []

crawled_link_list = []

for link in post_link_list[:num_page]:
    # 게시물 타이틀 수집
    driver.get(link)
    time.sleep(3)
    driver.implicitly_wait(3)
    soup = bs(driver.page_source, 'lxml')
    title = soup.title.text[34:]

    # 날짜 수집
    date = soup.find('time').get('datetime')[:10]

    # 인사이트 보기 버튼 클릭
    time.sleep( random.uniform(1, 3) )
    section = driver.find_element_by_css_selector('section.fmD5w')
    button = section.find_element_by_css_selector('button.sqdOP.yWX7d.y3zKF')
    button.click()

    driver.implicitly_wait(3) # 페이지 로딩 대기 시간
    time.sleep( random.uniform(2, 4) ) # 크롤링 차단 방지 랜덤하게 시간 기다리기

    # 조회수
    temp = driver.find_element_by_xpath('/html/body/div[8]/div/div/div/div[2]/div/div/div[2]/div/div/div[7]/div/div/div/div[3]/span[2]')
    if ',' in temp.text:
        temp = temp.text.replace(',','')
        num_view = int(temp)
    else:
        num_view = int(temp.text)

    # 좋아요, 댓글, 공유 수 정보
    temp = driver.find_elements_by_xpath('/html/body/div[8]/div/div/div/div[2]/div/div/div[2]/div/div/div[3]/div/div/div/div/div')

    # 좋아요 수
    if ',' in temp[0].text:
        num_like = int(temp[0].text.replace(',', ''))
    else:
        num_like = int(temp[0].text)
    # 댓글 수
    if ',' in temp[1].text:
        num_comment = int(temp[1].text.replace(',', ''))
    else:
        num_comment = int(temp[1].text)
    # 공유 수
    if ',' in temp[2].text:
        num_share = int(temp[2].text.replace(',', ''))
    else:
        num_share = int(temp[2].text)

    
    # 정보를 리스트에 저장
    date_list.append(date)
    title_list.append(title)
    num_view_list.append(num_view)
    num_like_list.append(num_like)
    num_comment_list.append(num_comment)
    num_share_list.append(num_share)
    crawled_link_list.append(link)

# 팔로워 수 리스트에 저장
num_follower_list = []
num_follower_list.append(num_follower)
for i in range(num_page-1):
    num_follower_list.append(0)

# chromedriver 종료
driver.quit()

<ipython-input-47-0b5ccef0caa1>:23: DeprecationWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  section = driver.find_element_by_css_selector('section.fmD5w')
C:\anaconda\lib\site-packages\selenium\webdriver\remote\webelement.py:484: UserWarning: find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead
  warnings.warn("find_element_by_css_selector is deprecated. Please use find_element(by=By.CSS_SELECTOR, value=css_selector) instead")
<ipython-input-47-0b5ccef0caa1>:31: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  temp = driver.find_element_by_xpath('/html/body/div[8]/div/div/div/div[2]/div/div/div[2]/div/div/div[7]/div/div/div/div[3]/span[2]')
<ipython-input-47-0b5ccef0caa1>:39: DeprecationWarning: find_elements_by_xpath is deprecated. Please use find_elements(by=By.XPATH, value=

## 수집한 데이터 개수 확인

In [48]:
# 수집한 자료 확인
print('날짜 리스트 원소 수 : ', len(date_list))
print('제목 리스트 원소 수 : ', len(title_list))
print('조회수 리스트 원소 수 : ', len(num_view_list))
print('좋아요 수 리스트 원소 수 : ', len(num_like_list))
print('공유 수 리스트 원소 수 : ', len(num_share_list))
print('댓글 수 리스트 원소 수 : ', len(num_comment_list))
print('팔로워 수 리스트 원소 수 : ', len(num_follower_list))
print('수집한 링크 리스트 원소 수 : ', len(crawled_link_list))

날짜 리스트 원소 수 :  30
제목 리스트 원소 수 :  30
조회수 리스트 원소 수 :  30
좋아요 수 리스트 원소 수 :  30
공유 수 리스트 원소 수 :  30
댓글 수 리스트 원소 수 :  30
팔로워 수 리스트 원소 수 :  30
수집한 링크 리스트 원소 수 :  30


## 데이터 셋 만들기

In [49]:
# 수집한 데이터를 데이터 셋으로 만들기
df = {'날짜':date_list, '제목':title_list, '조회수':num_view_list, '좋아요_수':num_like_list, '공유수':num_share_list, '댓글수':num_comment_list, '팔로워_수':num_follower_list[:30], '링크':crawled_link_list}
data = pd.DataFrame(df)
data

,날짜,제목,조회수,좋아요_수,공유수,댓글수,팔로워_수,링크
0,2022-03-23,A Beautiful Time to Become One with Nature in ...,75,15,0,0,9755,https://www.instagram.com/p/CbbxBIRvhi1/
1,2022-03-21,It is a place passed by only a few local buses...,181,21,0,0,0,https://www.instagram.com/p/CbXJ1rXPJdy/
2,2022-03-19,Global icon of video art 📺\nNam June Paik👍 \nY...,212,24,0,2,0,https://www.instagram.com/p/CbRd0niv2Vl/
3,2022-03-18,Gyeonggi Province is home to the Korea Nationa...,187,24,0,2,0,https://www.instagram.com/p/CbPIO4bPUQJ/
4,2022-03-15,"Have you heard of Empress Myeongseong, an icon...",202,24,0,0,0,https://www.instagram.com/p/CbHbw-uB5B3/
5,2022-03-12,Moran Station’s hottest restaurant with a stat...,235,26,0,0,0,https://www.instagram.com/p/Ca_cN1HPMlv/
6,2022-03-10,It’s a retro throwback \nNamchondong Mural Vil...,257,35,0,4,0,https://www.instagram.com/p/Ca6eIqfPoTV/
7,2022-03-08,A popular place in Seongnam City of Gyeonggi P...,270,25,0,0,0,https://www.instagram.com/p/Ca1mOngPNr7/
8,2022-03-04,Have you heard? (whispering)😝\nKorea’s first-e...,236,25,0,2,0,https://www.instagram.com/p/CarTSR8hpjk/
9,2022-03-02,A complete guide to unique cafés in Yongin Cit...,229,28,0,0,0,https://www.instagram.com/p/CamCGL3LSHX/


## 데이터 셋. 날짜로 슬라이싱

In [50]:
data2 = data[data['날짜'] >= '2022-01-01']
data2

,날짜,제목,조회수,좋아요_수,공유수,댓글수,팔로워_수,링크
0,2022-03-23,A Beautiful Time to Become One with Nature in ...,75,15,0,0,9755,https://www.instagram.com/p/CbbxBIRvhi1/
1,2022-03-21,It is a place passed by only a few local buses...,181,21,0,0,0,https://www.instagram.com/p/CbXJ1rXPJdy/
2,2022-03-19,Global icon of video art 📺\nNam June Paik👍 \nY...,212,24,0,2,0,https://www.instagram.com/p/CbRd0niv2Vl/
3,2022-03-18,Gyeonggi Province is home to the Korea Nationa...,187,24,0,2,0,https://www.instagram.com/p/CbPIO4bPUQJ/
4,2022-03-15,"Have you heard of Empress Myeongseong, an icon...",202,24,0,0,0,https://www.instagram.com/p/CbHbw-uB5B3/
5,2022-03-12,Moran Station’s hottest restaurant with a stat...,235,26,0,0,0,https://www.instagram.com/p/Ca_cN1HPMlv/
6,2022-03-10,It’s a retro throwback \nNamchondong Mural Vil...,257,35,0,4,0,https://www.instagram.com/p/Ca6eIqfPoTV/
7,2022-03-08,A popular place in Seongnam City of Gyeonggi P...,270,25,0,0,0,https://www.instagram.com/p/Ca1mOngPNr7/
8,2022-03-04,Have you heard? (whispering)😝\nKorea’s first-e...,236,25,0,2,0,https://www.instagram.com/p/CarTSR8hpjk/
9,2022-03-02,A complete guide to unique cafés in Yongin Cit...,229,28,0,0,0,https://www.instagram.com/p/CamCGL3LSHX/


## 데이터 셋. 인덱스 내림차순 정렬

In [51]:
# 인덱스 내림차순 정렬
final_data = data2.sort_index(ascending=False)
final_data

,날짜,제목,조회수,좋아요_수,공유수,댓글수,팔로워_수,링크
29,2022-01-01,🎊 Happy New Year! 🎊\nWe hope you have a happy ...,219,31,2,0,0,https://www.instagram.com/p/CYK_OtcFGPW/
28,2022-01-05,Enjoy your #winter more in Gyeonggi-do!\n\n#Ya...,284,41,1,1,0,https://www.instagram.com/p/CYV33FsPLHL/
27,2022-01-06,"Good walking path in Yangju-si, Gyeonggi-do🌳🌲\...",237,31,0,0,0,https://www.instagram.com/p/CYYeBR9v8_1/
26,2022-01-07,Gyeonggi Winter Camping❄️\n\nThe best camping ...,275,32,0,0,0,https://www.instagram.com/p/CYa1V4Yv1cp/
25,2022-01-13,Imjingak Pyeonghwa Nuri Park in Paju\n\nA plac...,301,27,0,2,0,https://www.instagram.com/p/CYqqLSpPg0L/
24,2022-01-14,Gwanggyo Lake Park\n\nA warm healing space tha...,1374,25,0,1,0,https://www.instagram.com/p/CYt6dZPv1vn/
23,2022-01-17,Hwadam Botanic Garden\n\n🌷Where is the best pl...,300,36,0,1,0,https://www.instagram.com/p/CY07xfKF0wg/
22,2022-01-20,🎹 Start the music!! 🎺🎺\n\nHow many museums in ...,219,28,1,0,0,https://www.instagram.com/p/CY8HnCJP5c1/
21,2022-01-24,Are you a big fan of theme parks?😍\n\nIntroduc...,401,26,0,0,0,https://www.instagram.com/p/CZGj2IOvVIf/
20,2022-01-26,Beautiful Suwon Hwaseong Fortress: A UNESCO Wo...,214,26,0,0,0,https://www.instagram.com/p/CZL99vIDzxe/


## 엑셀 파일로 출력

In [52]:
# 수집한 데이터 파일로 출력하기
file_name = str(datetime.today().strftime("%Y%m%d-%H%M%S"))
file_name = 'instagram_' + file_name + '.xlsx'
path = './crawling_data/instagram/'
final_data.to_excel(path+file_name, index=False, encoding='utf-8')

print(f"{file_name} 파일 생성 완료")

instagram_20220323-172450.xlsx 파일 생성 완료


# 코드 마지막